In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Импортирую необходимые библиотеки

In [3]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [47]:
import pandas as pd
import gzip
import json
import re
import gensim
import gensim.downloader
from tqdm import tqdm
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import BigramCollocationFinder
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

### Загружаю датасет отзывов на телефоны и аксессуары

In [5]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in tqdm(parse(path)):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('drive/MyDrive/reviews_Cell_Phones_and_Accessories_5.json.gz')

194439it [00:05, 33000.33it/s]


In [6]:
df

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4.0,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5.0,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5.0,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4.0,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5.0,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"
...,...,...,...,...,...,...,...,...,...
194434,A1YMNTFLNDYQ1F,B00LORXVUE,eyeused2loveher,"[0, 0]",Works great just like my original one. I reall...,5.0,This works just perfect!,1405900800,"07 21, 2014"
194435,A15TX8B2L8B20S,B00LORXVUE,Jon Davidson,"[0, 0]",Great product. Great packaging. High quality a...,5.0,Great replacement cable. Apple certified,1405900800,"07 21, 2014"
194436,A3JI7QRZO1QG8X,B00LORXVUE,Joyce M. Davidson,"[0, 0]","This is a great cable, just as good as the mor...",5.0,Real quality,1405900800,"07 21, 2014"
194437,A1NHB2VC68YQNM,B00LORXVUE,Nurse Farrugia,"[0, 0]",I really like it becasue it works well with my...,5.0,I really like it becasue it works well with my...,1405814400,"07 20, 2014"


Способы найти упоминания товаров в отзывах:
1. Взять готовый NER - вообще не требует никакой работы с нашей стороны, однако может являться не таким надежным, если будем применять к достаточно узким тематикам (может выделять не все слова)
2. Воспользоваться метаданными и выделить оттуда названия товаров - здесь есть много способов (как хороших, так и не очень) сделать это, однако почти все они будут не самыми надежными (хотя, вероятно, надежнее, чем простой NER), а в некоторых случаях будут требовать и ручной работы.
3. Связать ключевые понятия из текстов, взять их ближайших соседей - способ не самый надежный (опять же, на определенной тематике может работать лучше обычного NER), зато достаточно быстрый, не требует особой ручной работы.

Я выбрал третий способ, подключил к нему также нахождение похожих понятий из модели Glove, чтобы увеличить покрытие

In [37]:
known_words = {} #словарь для того, чтобы не лемматизировать одинаковые слова много раз
def preprocessing(text, stop_words, known_words):
    wnl = WordNetLemmatizer() #лемматизируем с помощью WordNetLemmatizer из NLTK
    lemmas = [] #сюда кладем все леммы в тексте
    regex_text = re.sub('[^\w\s]', '', text)
    regex_text = re.sub('[\d]', '', regex_text) #чистим текст от пунктуации, переносов строк и чисел
    regex_text = re.sub('\n', ' ', regex_text)
    for word in regex_text.split(' '):
        if word in stop_words: #добавляем леммы в словарь со встретившимися словами и в список лемм
            continue 
        elif word in known_words:
            lemmas.append(known_words[word])
        else:
            lemma = wnl.lemmatize(word.lower())
            lemmas.append(lemma)
            known_words[word] = lemma
    return lemmas

In [8]:
preprocessed = [preprocessing(line, stop_words, known_words) for line in tqdm(df['reviewText'])]

100%|██████████| 194439/194439 [01:03<00:00, 3042.00it/s]


In [9]:
model = gensim.models.Word2Vec(preprocessed, window=5)

In [31]:
wnl = WordNetLemmatizer()
our_vocab = set()
for word in model.wv.most_similar('phone', topn=8) + model.wv.most_similar('accessory', topn=8) + model.wv.most_similar('apple', topn=8) + model.wv.most_similar('android', topn=8) + model.wv.most_similar('smartphone', topn=8):
    our_vocab.add(wnl.lemmatize(word[0]))

In [32]:
print(our_vocab)
print(len(our_vocab))

{'extender', 'mfi', 'io', 'symbian', 'adaptor', 'touchwiz', 'platform', 'lightning', 'na', 'iphone', 'phoneit', 'lightening', 'licensed', 'powerhungry', 'wp', 'smartphones', 'face', 'smartphone', 'pda', 'o', 'kit', 'it', 'mygrid', 'tablet', 'aftermarket', 'duracell', 'smart', 'powermat', 'certified', 'device', 'ic', 'nonapple', 'webos', 'adapter', 'phablet', 'phonei', 'cellphone'}
37


Добавил к выделенным словам самые ближайшие к ним из glove-twitter-100 (при условии, что эти слова употреблялись в обзорах)

In [19]:
twitter_100 = gensim.downloader.load('glove-twitter-100')

In [33]:
all_words = set()
for line in preprocessed:
    for word in line:
        all_words.add(word)

In [35]:
twitter_vocab = set()
for word in our_vocab:
    if word in twitter_100:
      for another_word in twitter_100.most_similar(word):
          if word != wnl.lemmatize(another_word[0]) and wnl.lemmatize(another_word[0]) != "''" and wnl.lemmatize(another_word[0]) in all_words:
              twitter_vocab.add(wnl.lemmatize(another_word[0]))
              break

In [36]:
our_vocab = our_vocab.union(twitter_vocab)
print(our_vocab)
print(len(our_vocab))

{'samsung', 'fon', 'extender', 'mfi', 'io', 'cell', 'symbian', 'blacberry', 'adaptor', 'touchwiz', 'platform', 'lightning', 'na', 'pack', 'smartwatch', 'duracel', 'iphone', 'phoneit', 'lightening', 'licensed', 'powerhungry', 'protector', 'professional', 'wp', 'automotive', 'smartphones', 'face', 'smartphone', 'pda', 'ipad', 'o', 'notepro', 'that', 'kit', 'pd', 'it', 'md', 'mygrid', 'tablet', 'aftermarket', 'duracell', 'nokia', 'smart', 'system', 'powermat', 'e', 'certified', 'mo', 'device', 'ic', 'nonapple', 'webos', 'cool', 'look', 'poi', 'adapter', 'phablet', 'registered', 'enterprise', 'phonei', 'diagram', 'cellphone'}
62


In [38]:
all_texts = []
for line in preprocessed:
    all_texts += line

### Находим биграммы с полученными словами (как с левым, так и с правым соседом)

In [59]:
bigrams = []
for ind, elem in enumerate(all_texts):
    if elem in our_vocab and ind != 0 and ind + 1 != len(all_texts) and all_texts[ind - 1] != '' and all_texts[ind + 1] != '':
        bigrams.append([all_texts[ind - 1], all_texts[ind]])
        bigrams.append([all_texts[ind], all_texts[ind + 1]])

In [60]:
bigrams

[['they', 'look'],
 ['look', 'good'],
 ['phone', 'look'],
 ['look', 'stylish'],
 ['believe', 'that'],
 ['that', 'one'],
 ['screen', 'protector'],
 ['protector', 'i'],
 ['stay', 'look'],
 ['look', 'great'],
 ['described', 'it'],
 ['it', 'doesnt'],
 ['s', 'it'],
 ['it', 'double'],
 ['day', 'it'],
 ['it', 'doesnt'],
 ['purchase', 'it'],
 ['it', 'work'],
 ['cheaper', 'it'],
 ['it', 'definetly'],
 ['variety', 'it'],
 ['it', 'working'],
 ['stocked', 'it'],
 ['it', 'worked'],
 ['phone', 'that'],
 ['that', 'always'],
 ['two', 'it'],
 ['it', 'work'],
 ['cheap', 'adapter'],
 ['adapter', 'this'],
 ['phone', 'it'],
 ['it', 'came'],
 ['charge', 'ipad'],
 ['ipad', 'iohone'],
 ['usb', 'device'],
 ['device', 'if'],
 ['fine', 'cool'],
 ['cool', 'blue'],
 ['charge', 'iphone'],
 ['iphone', 'tablet'],
 ['iphone', 'tablet'],
 ['tablet', 'car'],
 ['work', 'iphone'],
 ['iphone', 'tablet'],
 ['iphone', 'tablet'],
 ['tablet', 'i'],
 ['charge', 'samsung'],
 ['samsung', 'iphone'],
 ['samsung', 'iphone'],
 ['ipho


### Ранжируем биграммы с помощью меры Дайса, PMI и Likelihood Ratio

In [64]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_documents(bigrams)
finder.apply_freq_filter(10)

In [65]:
pmi_score = finder.score_ngrams(bigram_measures.pmi)
print(pmi_score[:15])

[(('samsungs', 'touchwiz'), 11.438974169858154), (('pebble', 'smartwatch'), 10.77519292229591), (('touchwiz', 'interface'), 10.067005392471193), (('powermat', 'pmpb'), 9.731695085663084), (('theduracell', 'powermat'), 9.670294540998942), (('professional', 'photographer'), 9.464057146276673), (('powermat', 'gopower'), 9.444413133582614), (('astro', 'e'), 9.276658517826334), (('este', 'e'), 9.254221128706583), (('e', 'un'), 9.235362101455264), (('e', 'muy'), 9.173301133323015), (('mfi', 'certified'), 9.05108034587824), (('powermat', 'travelmat'), 8.92434016360548), (('kit', 'kat'), 8.578334980818841), (('powermat', 'powersnap'), 8.56177008422077)]


In [66]:
dice_score = finder.score_ngrams(bigram_measures.dice)
print(dice_score[:15])

[(('screen', 'protector'), 0.4991696111104919), (('cell', 'phone'), 0.3458168252407531), (('battery', 'pack'), 0.34012905639203217), (('samsung', 'galaxy'), 0.321307084505156), (('duracell', 'powermat'), 0.25127601099332547), (('lightning', 'cable'), 0.19265171322416402), (('mfi', 'certified'), 0.17563739376770537), (('smart', 'phone'), 0.15634229905879574), (('samsungs', 'touchwiz'), 0.14057507987220447), (('lightning', 'connector'), 0.1345227475468332), (('operating', 'system'), 0.1297484822202949), (('nokia', 'lumia'), 0.11386138613861387), (('that', 'said'), 0.11325818572195384), (('look', 'like'), 0.11104539025331105), (('apple', 'certified'), 0.10411357844921733)]


In [68]:
likelihood_ratio_score = finder.score_ngrams(bigram_measures.likelihood_ratio)
print(likelihood_ratio_score[:15])

[(('screen', 'protector'), 122623.76940031513), (('samsung', 'galaxy'), 41239.29570299865), (('it', 'it'), 38417.60537118369), (('cell', 'phone'), 31035.392412556008), (('battery', 'pack'), 24113.19336933007), (('it', 'device'), 20764.21552569272), (('it', 'iphone'), 18748.19444417088), (('it', 'protector'), 17095.71235657856), (('look', 'it'), 15755.975344616443), (('look', 'like'), 14114.543949190298), (('device', 'it'), 12859.688005558366), (('iphone', 'it'), 12710.161068158419), (('iphone', 's'), 12251.07987892353), (('protector', 'it'), 11074.995352738993), (('device', 'device'), 9313.555115064988)]


Кажется, что мера Дайса показывает лучший результат из трех

## Группируем коллокации

Группировка коллокаций для **меры Дайса**

In [69]:
groupped_dice = {word: [] for word in our_vocab}

In [70]:
for gram in dice_score:
  for word in gram[0]:
    if word in groupped_dice:
      groupped_dice[word].append(' '.join(gram[0]))
      break

In [79]:
printed = 0
for elem in groupped_dice:
  if printed != 5:
    if groupped_dice[elem] != []:
      print(elem)
      print('---')
      for collocation in groupped_dice[elem]:
        print(collocation)
      print('\n')
      printed += 1
  else:
    break

samsung
---
samsung galaxy
samsung note
samsung s
fit samsung
samsung battery
case samsung
charge samsung
original samsung
samsung phone
i samsung
use samsung
phone samsung
new samsung
the samsung
samsung product
oem samsung
samsung charger
samsung oem
battery samsung
work samsung
samsung epic
using samsung
bought samsung
charger samsung
samsung tablet
samsung g
like samsung
used samsung
one samsung
well samsung
genuine samsung
my samsung
samsung tab
s samsung
samsung focus
got samsung
great samsung
purchased samsung
samsung infuse
samsung captivate
cover samsung
old samsung
samsung i
samsung cable
wife samsung
came samsung
perfectly samsung
cable samsung
stock samsung
official samsung
hold samsung
samsung smartphone
samsung android
product samsung
verizon samsung
get samsung
for samsung
say samsung
samsung device
samsung logo
att samsung
real samsung
made samsung
charging samsung
samsung flip
charged samsung
samsung cell
samsung smart
samsung nexus
sprint samsung
better samsung
love s

Группировка коллокаций для **PMI**

In [80]:
groupped_pmi = {word: [] for word in our_vocab}

In [81]:
for gram in pmi_score:
  for word in gram[0]:
    if word in groupped_pmi:
      groupped_pmi[word].append(' '.join(gram[0]))
      break

In [82]:
printed = 0
for elem in groupped_pmi:
  if printed != 5:
    if groupped_pmi[elem] != []:
      print(elem)
      print('---')
      for collocation in groupped_pmi[elem]:
        print(collocation)
      print('\n')
      printed += 1
  else:
    break

samsung
---
samsung sview
samsung exynos
samsung wep
samsung intercept
samsung blackjack
samsung epic
samsung galaxy
samsung stratosphere
samsung galazy
samsung instinct
samsung fascinate
samsung infuse
samsung captivate
samsung hm
samsung exhibit
samsung focus
genuine samsung
hyperion samsung
authentic samsung
official samsung
xoom samsung
samsung skyrocket
samsung note
samsung tab
samsung illusion
samsung sii
samsung flip
samsung vibrant
samsung branded
tmobile samsung
original samsung
stock samsung
samsung siii
sprint samsung
sgp samsung
samsung moment
samsung logo
oem samsung
samsung grade
samsung h
samsung oem
samsung iii
called samsung
samsung t
samsung gear
switched samsung
att samsung
samsung website
factory samsung
verizon samsung
purchased samsung
samsung droid
samsung site
samsung wireless
upgraded samsung
real samsung
armor samsung
samsung earphone
samsung name
believe samsung
husband samsung
samsung qi
receiver samsung
old samsung
bought samsung
samsung htc
samsung earbuds

Группировка коллокаций для **Likelihood**

In [83]:
groupped_likelihood = {word: [] for word in our_vocab}

In [84]:
for gram in likelihood_ratio_score:
  for word in gram[0]:
    if word in groupped_likelihood:
      groupped_likelihood[word].append(' '.join(gram[0]))
      break

In [85]:
printed = 0
for elem in groupped_likelihood:
  if printed != 5:
    if groupped_likelihood[elem] != []:
      print(elem)
      print('---')
      for collocation in groupped_likelihood[elem]:
        print(collocation)
      print('\n')
      printed += 1
  else:
    break

samsung
---
samsung galaxy
samsung it
samsung iphone
samsung look
samsung device
samsung note
samsung samsung
original samsung
screen samsung
samsung s
samsung epic
oem samsung
genuine samsung
samsung oem
samsung adapter
samsung infuse
samsung captivate
samsung focus
samsung tab
fit samsung
samsung cell
samsung battery
official samsung
new samsung
stock samsung
bought samsung
samsung like
samsung exhibit
samsung product
samsung great
galaxy samsung
samsung case
samsung flip
samsung fascinate
samsung wep
samsung charge
samsung i
purchased samsung
samsung hm
samsung logo
samsung fit
sprint samsung
tmobile samsung
samsung stratosphere
samsung vibrant
authentic samsung
old samsung
att samsung
use samsung
samsung branded
using samsung
samsung work
verizon samsung
samsung intercept
case samsung
my samsung
samsung tablet
real samsung
good samsung
charge samsung
xoom samsung
samsung charger
used samsung
samsung the
samsung wireless
samsung g
wife samsung
hyperion samsung
samsung also
samsung w